In [1]:
%reload_ext autoreload
%autoreload 2

In [31]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
from torch_geometric.nn import GraphSAGE, MessagePassing
from opacus import PrivacyEngine
from torch.utils.data import TensorDataset, DataLoader
from privacy import NoisySGD
from data import PoissonDataLoader, NeighborSampler, Dataset
from console import console
from torch_geometric.datasets import RandomPartitionGraphDataset
from torch_geometric.utils import sort_edge_index
from torch_sparse import SparseTensor
from torch_geometric.data import LightningNodeData
from methods import GraphSAGEModel
from scipy.stats import hypergeom
from methods import GAP
from scipy.io import loadmat
from ogb.nodeproppred import PygNodePropPredDataset

console.width = 150

In [4]:
mat = loadmat('datasets/facebook/UIllinois20/raw/UIllinois20.mat')
mat

{'__header__': b'MATLAB 5.0 MAT-file, Platform: MACI64, Created on: Fri Feb 11 20:03:31 2011',
 '__version__': '1.0',
 '__globals__': [],
 'A': <30809x30809 sparse matrix of type '<class 'numpy.float64'>'
 	with 2528856 stored elements in Compressed Sparse Column format>,
 'local_info': array([[   1,    2,    0, ...,   37, 2008, 7008],
        [   1,    1,  157, ...,    0, 2006, 5784],
        [   1,    2,   78, ...,  110, 2006, 6265],
        ...,
        [   1,    2,  209, ...,  198, 2009, 5456],
        [   1,    2,   78, ...,    0, 2006, 5638],
        [   1,    1,    0, ...,   70, 2009, 6159]], dtype=uint16)}

In [5]:
features = pd.DataFrame(mat['local_info'][:, :-1], columns=['status', 'gender', 'major', 'minor', 'housing', 'year'])
features

,status,gender,major,minor,housing,year
0,1,2,0,0,37,2008
1,1,1,157,0,0,2006
2,1,2,78,0,110,2006
3,1,0,75,0,0,2007
4,1,2,77,67,55,2007
...,...,...,...,...,...,...
30804,1,1,45,157,197,2008
30805,1,2,78,131,0,2006
30806,1,2,209,0,198,2009
30807,1,2,78,193,0,2006


In [6]:
features.groupby('year').count()

,status,gender,major,minor,housing
year,,,,,
0,4006,4006,4006,4006,4006
1900,2,2,2,2,2
1903,1,1,1,1,1
1949,1,1,1,1,1
1968,1,1,1,1,1
1969,2,2,2,2,2
1970,1,1,1,1,1
1971,1,1,1,1,1
1972,2,2,2,2,2


In [7]:
data = Dataset('facebook').load(True)

Processing...
Done!


[18:02:35] INFO                                                         dataset stats                                                      ]8;id=453779;file:///remote/idiap.svm/user.active/sajadmanesh/Github/GAP/data.py\data.py]8;;\:]8;id=745161;file:///remote/idiap.svm/user.active/sajadmanesh/Github/GAP/data.py#230\230]8;;\
                     ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────             
                      name       nodes    edges       features   classes   mean degree   median degree   train/val/test     baseline (%)              
                                                                                                         (%)                                          
                     ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────             
                      facebook   26,406   2,117,924   501        6         80.21         62.0            75.01 / 9.98 /     22.59                     
                                                                                                         15.00                                        
                     ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [8]:
data.y[data.train_mask].unique(return_counts=True)[1].min()

(tensor([0, 1, 2, 3, 4, 5]), tensor([ 798, 2878, 3400, 4015, 4549, 4168]))

In [9]:
data = Dataset('reddit').load(True)

[18:05:41] INFO                                                         dataset stats                                                      ]8;id=793755;file:///remote/idiap.svm/user.active/sajadmanesh/Github/GAP/data.py\data.py]8;;\:]8;id=984881;file:///remote/idiap.svm/user.active/sajadmanesh/Github/GAP/data.py#230\230]8;;\
                     ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────             
                      name     nodes     edges         features   classes   mean degree   median degree   train/val/test    baseline (%)              
                                                                                                          (%)                                         
                     ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────             
                      reddit   232,965   114,615,892   602        41        491.99        230.0           65.86 / 10.23 /   14.83                     
                                                                                                          23.91                                       
                     ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [14]:
data.y[data.train_mask].unique(return_counts=True)[1].sort().values

tensor([  236,   721,  1065,  1096,  1131,  1152,  1455,  1481,  1567,  1647,
         1875,  1881,  2009,  2107,  2226,  2276,  2360,  2384,  2795,  2803,
         2826,  2841,  2855,  2884,  3135,  3224,  3273,  3336,  3371,  3376,
         3851,  4056,  5311,  7072,  7280,  7679,  8129,  8457,  9103, 10589,
        16516])

In [48]:
data = Dataset('amazon').load(True)

[09:19:04] INFO                                                         dataset stats                                                      ]8;id=294235;file:///remote/idiap.svm/user.active/sajadmanesh/Github/GAP/data.py\data.py]8;;\:]8;id=904657;file:///remote/idiap.svm/user.active/sajadmanesh/Github/GAP/data.py#230\230]8;;\
                     ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────             
                      name     nodes       edges         features   classes   mean degree   median degree   train/val/te…   baseline (%)              
                                                                                                            (%)                                       
                     ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────             
                      amazon   2,400,608   123,718,024   100        47        51.54         27.0            8.19 / 1.64 /   26.59                     
                                                                                                            90.18                                     
                     ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [55]:
data.y.unique(return_counts=True)[1].sort().values

tensor([     1,      2,      9,     29,     37,     44,     60,     91,    154,
           245,    253,    417,    512,    512,    546,    555,    620,    866,
          1331,   1497,   1868,   3004,   3052,   3606,  17240,  22416,  26858,
         29553,  32314,  40469,  41921,  43801,  48299,  51654,  66514,  80320,
         82681,  99995, 108651, 110488, 112338, 113308, 131470, 145426, 157591,
        169628, 648362])

In [54]:
data.y[data.train_mask].unique(return_counts=True)[1].sort().values

tensor([    1,     1,     1,     1,     2,     2,     3,     3,     3,     4,
            8,    25,    33,    64,    75,    84,    93,   152,   200,   287,
          461,   526,   662,  1237,  1313,  2047,  2515,  2755,  3340,  3932,
         4245,  4311,  6266, 10756, 10926, 10941, 11332, 12290, 13151, 15305,
        16600, 60618])